In [1]:
from datetime import datetime
import numpy as np
import pandas as pd

In [2]:
# Read Country table to get foreign key value
df_ref = pd.read_csv("./cleaned/Country.csv")
df_ref.head()

,id,name,population
0,1,Afghanistan,39074280
1,2,Albania,2877239
2,3,Algeria,43984569
3,4,American Samoa,55169
4,5,Andorra,77287


In [3]:
# Read original CSV
date_parser = lambda x: datetime.strptime(x, '%Y-%m-%d')
df_in = pd.read_csv("./raw/covid-19-world-cases-deaths-testing.csv", parse_dates=['date'], date_parser=date_parser)
df_in.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality
0,AFG,Asia,Afghanistan,2020-02-24,5.0,5.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
1,AFG,Asia,Afghanistan,2020-02-25,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
2,AFG,Asia,Afghanistan,2020-02-26,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
3,AFG,Asia,Afghanistan,2020-02-27,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN
4,AFG,Asia,Afghanistan,2020-02-28,5.0,0.0,NaN,NaN,NaN,NaN,...,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.511,NaN


In [4]:
# Initialize new dataframe
columns = [
    "record_date",
    "daily_tests",
    "daily_positive_cases",
    "daily_deaths",
    "daily_vaccination",
    "country",
    "country_id"
]
df_out = pd.DataFrame([], columns=columns)
df_out.head()

,record_date,daily_tests,daily_positive_cases,daily_deaths,daily_vaccination,country,country_id


In [5]:
for country in df_in['location'].unique():
    country_ids = df_ref[df_ref['name'] == country]["id"] 
    if(len(country_ids) == 0):
        continue
    country_id = country_ids.iloc[0]
    print('[INFO] Country_id: {}, Country: {}'.format(country_id, country))
    # Initialize total vaccination count for each country
    total_vaccination_count = None
    # Filter rows of any particular country
    df_country = df_in[df_in['location'] == country].sort_values(by=['date'])
    for i in range(len(df_country)):
        current = df_country.iloc[i]
        # date, daily_tests, daily_positive_count, daily_deaths are directly taken from the current row values
        record_date = current["date"]
        daily_tests = current["new_tests"]
        daily_positive_cases = current["new_cases"]
        daily_deaths = current["new_deaths"]
        # Calculate daily vaccination count using total vaccination count
        # If it's the first day, we do not have previous day's record
        # Take the maximum value if both total_vaccination and new_vaccination values are known
        # Else, take the total vaccination count if it's known
        # Else, take the daily vaccination count if it's known
        # Else, daily vaccination count is unknown
        if(i == 0):
            if((not np.isnan(current["total_vaccinations"])) and (not np.isnan(current["new_vaccinations"]))):
                daily_vaccination = max(current["total_vaccinations"], current["new_vaccinations"])
                total_vaccination_count = current["total_vaccinations"]
            elif(not np.isnan(current["total_vaccinations"])):
                daily_vaccination = current["total_vaccinations"]
                total_vaccination_count = current["total_vaccinations"]
            elif(not np.isnan(current["new_vaccinations"])):
                daily_vaccination = current["new_vaccinations"]
            else:
                daily_vaccination = None
        # For all other days, we have previous day's record
        # If we know total vaccination count of current day, previous, and also the daily vaccination count of the current day (I),
        # Calculate the daily vaccination count (II) by subtracting previous day total vaccination count from the current day total vaccination count,
        # And find out the maximum value between I and II.
        # Else, If total vaccination count for both current and previous days are known,
        # Calculate the daily vaccination count by subtracting previous day total vaccination count from the current day total vaccination count 
        # Else, If total vaccination count of only current day is known,
        # Calculate the daily vaccination count by subtracting total vaccination count of the country from current day's total vaccination count
        # Else, take the daily vaccination count if it's known
        # Else, daily vaccination count is unknown
        else:
            previous = df_country.iloc[i - 1]
            if(((not np.isnan(current["total_vaccinations"])) and (not np.isnan(previous["total_vaccinations"]))) and
               (not np.isnan(current["new_vaccinations"]))):
                daily_vaccination = max(
                    current["total_vaccinations"] - previous["total_vaccinations"],
                    current["new_vaccinations"]
                )
                total_vaccination_count = current["total_vaccinations"]
            elif((not np.isnan(current["total_vaccinations"])) and (not np.isnan(previous["total_vaccinations"]))):
                daily_vaccination = current["total_vaccinations"] - previous["total_vaccinations"]
                total_vaccination_count = current["total_vaccinations"]
            elif((not np.isnan(current["total_vaccinations"])) and (total_vaccination_count is not None)):
                daily_vaccination = current["total_vaccinations"] - total_vaccination_count
                total_vaccination_count = current["total_vaccinations"]
            elif(not np.isnan(current["total_vaccinations"])):
                daily_vaccination = current["total_vaccinations"]
                total_vaccination_count = current["total_vaccinations"]
            elif(not np.isnan(current["new_vaccinations"])):
                daily_vaccination = current["new_vaccinations"]
            else:
                daily_vaccination = None
                
        new_row = {
            "record_date": record_date,
            "daily_tests": daily_tests,
            "daily_positive_cases": daily_positive_cases,
            "daily_deaths": daily_deaths,
            "daily_vaccination": daily_vaccination,
            "country": country,
            "country_id": country_id
        }
        df_out = df_out.append(new_row, ignore_index=True)

[INFO] Country_id: 1, Country: Afghanistan
[INFO] Country_id: 2, Country: Albania
[INFO] Country_id: 3, Country: Algeria
[INFO] Country_id: 5, Country: Andorra
[INFO] Country_id: 6, Country: Angola
[INFO] Country_id: 7, Country: Anguilla
[INFO] Country_id: 8, Country: Antigua and Barbuda
[INFO] Country_id: 9, Country: Argentina
[INFO] Country_id: 10, Country: Armenia
[INFO] Country_id: 11, Country: Aruba
[INFO] Country_id: 12, Country: Australia
[INFO] Country_id: 13, Country: Austria
[INFO] Country_id: 14, Country: Azerbaijan
[INFO] Country_id: 15, Country: Bahamas
[INFO] Country_id: 16, Country: Bahrain
[INFO] Country_id: 17, Country: Bangladesh
[INFO] Country_id: 18, Country: Barbados
[INFO] Country_id: 19, Country: Belarus
[INFO] Country_id: 20, Country: Belgium
[INFO] Country_id: 21, Country: Belize
[INFO] Country_id: 22, Country: Benin
[INFO] Country_id: 23, Country: Bermuda
[INFO] Country_id: 24, Country: Bhutan
[INFO] Country_id: 25, Country: Bolivia
[INFO] Country_id: 26, Coun

[INFO] Country_id: 214, Country: Tunisia
[INFO] Country_id: 215, Country: Turkey
[INFO] Country_id: 216, Country: Turkmenistan
[INFO] Country_id: 217, Country: Turks and Caicos Islands
[INFO] Country_id: 218, Country: Tuvalu
[INFO] Country_id: 220, Country: Uganda
[INFO] Country_id: 221, Country: Ukraine
[INFO] Country_id: 222, Country: United Arab Emirates
[INFO] Country_id: 223, Country: United Kingdom
[INFO] Country_id: 224, Country: United States
[INFO] Country_id: 225, Country: Uruguay
[INFO] Country_id: 226, Country: Uzbekistan
[INFO] Country_id: 227, Country: Vanuatu
[INFO] Country_id: 228, Country: Vatican
[INFO] Country_id: 229, Country: Venezuela
[INFO] Country_id: 230, Country: Vietnam
[INFO] Country_id: 231, Country: Wallis and Futuna
[INFO] Country_id: 233, Country: Yemen
[INFO] Country_id: 234, Country: Zambia
[INFO] Country_id: 235, Country: Zimbabwe


In [6]:
# Remove aggregated data
df_out = df_out.drop(df_out[df_out["country_id"] == "World"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "International"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "Africa"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "Asia"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "Europe"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "European Union"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "North America"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "Oceania"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "South America"].index)

# Remove countries data for which population is unknown
df_out = df_out.drop(df_out[df_out["country_id"] == "Jersey"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "Kosovo"].index)
df_out.drop(df_out[df_out["country_id"] == "Northern Cyprus"].index)
df_out = df_out.drop(df_out[df_out["country_id"] == "Pitcairn"].index)

In [7]:
df_out

,record_date,daily_tests,daily_positive_cases,daily_deaths,daily_vaccination,country,country_id
0,2020-02-24,NaN,5.0,NaN,NaN,Afghanistan,1
1,2020-02-25,NaN,0.0,NaN,NaN,Afghanistan,1
2,2020-02-26,NaN,0.0,NaN,NaN,Afghanistan,1
3,2020-02-27,NaN,0.0,NaN,NaN,Afghanistan,1
4,2020-02-28,NaN,0.0,NaN,NaN,Afghanistan,1
...,...,...,...,...,...,...,...
109128,2021-09-10,2411.0,107.0,11.0,54428.0,Zimbabwe,235
109129,2021-09-11,2166.0,57.0,4.0,52457.0,Zimbabwe,235
109130,2021-09-12,2035.0,49.0,2.0,NaN,Zimbabwe,235
109131,2021-09-13,NaN,130.0,5.0,43451.0,Zimbabwe,235


In [8]:
# Rename some of the country names to match with the entries in Country table
df_out.loc[df_out.country_id == 'Bonaire Sint Eustatius and Saba', 'country_id'] = 'Caribbean Netherlands'
df_out.loc[df_out.country_id == 'Micronesia (country)', 'country_id'] = 'Micronesia'
df_out.loc[df_out.country_id == 'Sint Maarten (Dutch part)', 'country_id'] = 'Sint Maarten'

In [9]:
df_out

,record_date,daily_tests,daily_positive_cases,daily_deaths,daily_vaccination,country,country_id
0,2020-02-24,NaN,5.0,NaN,NaN,Afghanistan,1
1,2020-02-25,NaN,0.0,NaN,NaN,Afghanistan,1
2,2020-02-26,NaN,0.0,NaN,NaN,Afghanistan,1
3,2020-02-27,NaN,0.0,NaN,NaN,Afghanistan,1
4,2020-02-28,NaN,0.0,NaN,NaN,Afghanistan,1
...,...,...,...,...,...,...,...
109128,2021-09-10,2411.0,107.0,11.0,54428.0,Zimbabwe,235
109129,2021-09-11,2166.0,57.0,4.0,52457.0,Zimbabwe,235
109130,2021-09-12,2035.0,49.0,2.0,NaN,Zimbabwe,235
109131,2021-09-13,NaN,130.0,5.0,43451.0,Zimbabwe,235


In [10]:
df_out.to_csv("./cleaned/Country_covid_data.csv", index=False)